# Classifying Traffic Signs with Tensorflow

We would like to be able to build a system that can inspect our traffic signs: Are they where they should be and are those the right types. For our first step we will build a classifier that can classify a photo of a sign.

We have a set of hand tagged photo's of traffic signs we borrowed from https://medium.com/@waleedka/traffic-sign-recognition-with-tensorflow-629dffc391a6

In [1]:
#Let's inspect some.
#TODO

## Reading the data

In [15]:
print(getwd());
library("png");
im = readPNG("db/plaatjes/00000/1.png");
print(im);

[1] "/home/stijn/Documents/bordklassificatie"
, , 1

             [,1]       [,2]       [,3]       [,4]       [,5]       [,6]
  [1,] 0.10196078 0.25098039 0.23529412 0.22352941 0.29019608 0.34509804
  [2,] 0.10980392 0.19215686 0.19607843 0.20392157 0.25098039 0.31372549
  [3,] 0.11764706 0.13333333 0.16078431 0.18431373 0.21568627 0.27843137
  [4,] 0.10588235 0.11372549 0.14117647 0.16078431 0.18039216 0.22745098
  [5,] 0.09411765 0.10980392 0.12941176 0.13725490 0.14901961 0.18431373
  [6,] 0.09803922 0.10196078 0.11372549 0.12156863 0.13333333 0.14509804
  [7,] 0.10980392 0.10196078 0.09803922 0.10980392 0.12549020 0.12156863
  [8,] 0.11764706 0.10588235 0.09803922 0.09803922 0.10588235 0.10588235
  [9,] 0.12549020 0.11372549 0.10588235 0.10588235 0.10980392 0.11372549
 [10,] 0.11764706 0.10196078 0.10196078 0.14901961 0.19215686 0.16862745
 [11,] 0.11372549 0.10588235 0.12156863 0.20784314 0.28235294 0.22745098
 [12,] 0.11764706 0.14117647 0.18431373 0.26666667 0.32549020 0.2627451

Now we read in the data. We build a df data containing a colum labels (classnumbers) and column images containing flattend lists of normalized pixel values.

In [25]:
library("png");
source("functions.r");
if(file.exists("db/training/train.rds") & file.exists("db/training/test.rds")){
    train <- readRDS("db/training/train.rds")
    test <- readRDS("db/training/test.rds")
} else {
    #make a df of labels and images
    data = load_data("db/plaatjes");

    # Build a set of 90% per label
    comps <- levels(df$labels)
    trn <- sample(comps, length(comps)*0.7)
    data.train <- subset(df, labels %in% trn)
    data.test <- subset(df, !(labels %in% trn))
    
    #Save for future usage
    saveRDS(data.train, file = 'db/training/train.rds');
    saveRDS(data.test, file = 'db/testing/test.rds');
}

Warning message:
“NAs introduced by coercion”

[1] 107 108   3
[1] 100 100


ERROR: Error in aperm.default(image, c(3, 2, 1)): 'perm' is of wrong length 3 (!= 2)


## SMOTE (make more data)

Alright! Now we have a separate train and test set. But what if we could have more images? Let's scale our train set by adding some artificial new samples (SMOTING)

In [12]:
require(feather)

#We write the data to chunks to keep the operations in memory
write_feather(as.data.frame(train), path = paste("db/training/delen/deel0.fe", sep =''))
for(i in 1:100){#we want a hunderd times as many images
    #bring the noise  
    data.train$image = data.train$image+ rnorm(data.train$image, mean = 0, sd = 0.05) 
    #normalize the image (again)
    data.train$image = normalize_image( data.train$image);
    #apply a random rotation in the 3d sphere
    data.train$image = rotate3d(data.train$image)
    write_feather(totaal, path = paste("db/training/delen/deel", i ,".fe", sep =''))
}

Loading required package: feather
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘feather’”

ERROR: Error in eval(expr, envir, enclos): could not find function "write_feather"


## Making the DCNN

Now that we have data we will dfeine a DCNN to train a validate

In [ ]:
library(tensorflow);

#Convert the input to matrixes
train = as.matrix(data.train$images)
test  = as.matrix(data.test$images)
train_labels =as.matrix(data.test$labels)
test_labels = as.matrix(data.test$labels)

vector_train = c()
vector_test = c() 

max_accuracy = 0.97
sess <- tf$InteractiveSession()

###################### HET NETWERK

#input
x <- tf$placeholder(tf$float32, shape(NULL, as.integer(100)*as.integer(100)*as.integer(3)))
#target values
y_ <- tf$placeholder(tf$float32, shape(NULL, as.integer(ncol(train_labels))))
x_in = tf$reshape(x, shape(-1L,as.integer(100)*as.integer(100)*as.integer(3)))

# eerste convolutie laag
#maak gewichten en biasses
W_conv1 <- weight_variable(shape(5L, 5L, kanalen, 40L), 'W_conv1')
b_conv1 <- bias_variable(shape(40L), 'b_conv1')

h_conv1 <- tf$nn$relu(conv2d(x_in, W_conv1) + b_conv1)
h_pool1 <- max_pool_2x2(h_conv1)

#tweede convolutie laag
W_conv2 <- weight_variable(shape = shape(5L, 5L, 40L, 64L), 'W_conv2')
b_conv2 <- bias_variable(shape = shape(64L), 'b_conv2')

h_conv2 <- tf$nn$relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 <- max_pool_2x2(h_conv2)

#derde convolutielaag
W_conv3 <- weight_variable(shape = shape(5L, 5L, 64L, 64L), 'W_conv2')
b_conv3 <- bias_variable(shape = shape(64L), 'b_conv2')

h_conv3 <- tf$nn$relu(conv2d(h_pool2, W_conv3) + b_conv3)

# eerste connected laag
W_fc1 <- weight_variable(shape( (hoogte*hoogte)/16 * 64L, 1024L), 'W_fc1')
b_fc1 <- bias_variable(shape(1024L), 'b_fc1')

#maak tensor vlak
h_conv3_flat <- tf$reshape(h_conv3, shape(-1L, (hoogte*hoogte)/16 *64L))
#bereken output
h_fc1 <- tf$nn$relu(tf$matmul(h_conv3_flat, W_fc1) + b_fc1)

#droupout rate eerste connected layer
keep_prob <- tf$placeholder(tf$float32)
h_fc1_drop <- tf$nn$dropout(h_fc1, keep_prob)

#output laag
W_fc2 <- weight_variable(shape(1024L, clas), 'W_fc2')
b_fc2 <- bias_variable(shape(clas), 'b_fc2')
y_conv <- tf$nn$softmax(tf$matmul(h_fc1_drop, W_fc2) + b_fc2)

#################### einde netwerk

# evaluation 
cross_entropy <- tf$reduce_mean(-tf$reduce_sum(y_ * tf$log( tf$clip_by_value(y_conv, 1e-10,1 )), reduction_indices=1L))
#train met de Adam optimizer learning rate 1e-4 en probeerd de cross entropy te minimaliseren
lrate <- tf$placeholder(tf$float32)
train_step <- tf$train$AdamOptimizer(lrate)$minimize(cross_entropy)

correct_prediction <- tf$equal(tf$argmax(y_conv, 1L), tf$argmax(y_, 1L))
accuracy <- tf$reduce_mean(tf$cast(correct_prediction, tf$float32))

#initialiseer the network
sess$run(tf$global_variables_initializer())

## Training and evaluate the network

In [ ]:
for (i in 1:200000) {
  #lees 50 random plaatjes in
  samp = sample( dim(train)[1] , 50 )
  #train

  #valideer om de 100 keer hoe het gaat
  if (i %% 100 == 0) {
    #evaluate training accuracy
    train_accuracy <- accuracy$eval(feed_dict = dict(      x = train  , y_ = train_labels, keep_prob = 1.0))
    print( paste('step' , i , 'training accuracy' , train_accuracy) )
    
    #evaluate test accuracy
    test_accuracy <- accuracy$eval(feed_dict = dict(  x = test, y_ = test_labels, keep_prob = 1.0))
    print( paste("test accuracy", test_accuracy) )
    
    #sla de accuracies op
    train_accuracies = c(vector_train, train_accuracy)
    test_accuracies = c(vector_test, test_accuracy)
    
    if(test_accuracy > max_accuracy  ){
      save_network();
      max_accuracy = test_accuracy
      print('new record')
    }
    
    #Na 5000 train steps begin met andere bestanden
    if(i>5000){
    n = sample(0:99, 1)
    file = paste("db/training/delen/deel", n, '.fe' , sep='')
    train = read_feather(file)
    train = as.matrix(train)
    }
  }
  train_step$run(feed_dict = dict(x = train[samp,] , y_ = train_labels[samp,], keep_prob = 0.2, lrate = 1e-4))
}

#sla het accuracy verloop op
saveRDS(test_accuracies, 'db/neuraalnet/vector_test.rds')
saveRDS(train_accuracies, 'db/neuraalnet/vector_train.rds')